In [41]:
import ibis
from ibis import _
import ibis.selectors as s
import pandas as pd
import requests

con = ibis.duckdb.connect()

url = "https://raw.githubusercontent.com/boettiger-lab/wetlands/refs/heads/main/maplibre/wetland-colormap.json"

# Read JSON with requests and reshape with pandas
response = requests.get(url)
data = response.json()

# Convert to long format
rows = []
for key, value in data.items():
    # Unnest the color array
    color = value.get('color', [])
    rows.append({
        'id': key,
        'name': value.get('name'),
        'description': value.get('description'),
        'category': value.get('category'),
        'r': color[0] if len(color) > 0 else None,
        'g': color[1] if len(color) > 1 else None,
        'b': color[2] if len(color) > 2 else None,
        'a': color[3] if len(color) > 3 else None
    })

wetlands_df = pd.DataFrame(rows)
wetlands = ibis.memtable(wetlands_df)
wetlands.to_csv("maplibre/category_codes.csv")

In [47]:
# Show a sample of the data
wetlands.rename(Z = 'id').drop(['r', 'g', 'b', 'a']).to_csv("maplibre/category_codes.csv")